In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

In [ ]:
# Paths
input_dir = '/content/drive/MyDrive/CBIR_ENDSEM/images'
output_dir = '/content/drive/MyDrive/CBIR_ENDSEM/preprocessed'
os.makedirs(output_dir, exist_ok=True)
domains = ['Natural', 'brainMRI', 'Paintings']

for domain in domains:
    os.makedirs(os.path.join(output_dir, domain), exist_ok=True)

In [ ]:
def preprocess_natural(img):
    # Preserve aspect ratio, color enhancement, and normalization
    h, w = img.shape[:2]
    target_size = 256

    # Aspect ratio resize
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Pad to 256x256
    delta_w, delta_h = target_size - new_w, target_size - new_h
    top, bottom = delta_h // 2, delta_h - delta_h // 2
    left, right = delta_w // 2, delta_w - delta_w // 2
    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    # Convert to LAB and apply CLAHE on L channel
    lab = cv2.cvtColor(padded, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Gamma correction
    gamma = 1.1
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in np.arange(256)]).astype("uint8")
    corrected = cv2.LUT(enhanced, table)

    return corrected

In [ ]:
def preprocess_mri(img):
    # Convert to grayscale and normalize intensity
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    norm = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    return norm

In [ ]:
def preprocess_paintings(img):
    # Enhance color and edges
    img = cv2.resize(img, (256, 256))
    img = cv2.convertScaleAbs(img, alpha=1.1, beta=10)
    edges = cv2.Canny(img, 80, 150)

    # Merge edges for pattern emphasis
    edges_colored = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    enhanced = cv2.addWeighted(img, 0.8, edges_colored, 0.2, 0)
    return enhanced

In [ ]:
def process_domain(domain, preprocess_fn):
    input_path = os.path.join(input_dir, domain)
    output_path = os.path.join(output_dir, domain)

    for filename in tqdm(os.listdir(input_path), desc=f"Processing {domain}"):
        if filename.lower().endswith(('.jpg', '.png', '.jpeg', '.bmp')):
            img_path = os.path.join(input_path, filename)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Could not read {filename}")
                continue

            processed = preprocess_fn(img)
            save_path = os.path.join(output_path, filename)
            cv2.imwrite(save_path, processed)

process_domain('Natural', preprocess_natural)
process_domain('brainMRI', preprocess_mri)
process_domain('Paintings', preprocess_paintings)

print("\nPreprocessing complete)